# Lucas-Kanade Optical Flow

 The Lucas-Kanade method estimates the motion between two consecutive frames. It assumes that the motion is small and constant within a neighborhood of each pixel. The optical flow constraint equation is:
 
 $$ I_x u + I_y v + I_t = 0 $$
 
 where:
 - $ I_x $ and $ I_y $ are the spatial image gradients.
 - $ I_t $ is the temporal image gradient.
 - $ u $ and $ v $ are the optical flow vectors.

 The method solves this equation for each pixel using a least squares approach over a window of pixels.

### Step 1: Import Libraries

In [25]:

import numpy as np
import cv2 as cv
import argparse

### Step 2: Set Up Argument Parser
- The `initialize_video_capture` function initializes video capture from the specified video file. It checks if the video file is opened successfully.


In [26]:
def initialize_video_capture(video_path):
    """Initialize video capture from the specified video file."""
    cap = cv.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file.")
    return cap

### Step 3: Set Parameters
- The `set_feature_params` function defines parameters for Shi-Tomasi corner detection, which helps in identifying good features to track. The `set_lk_params` function sets parameters for the Lucas-Kanade optical flow algorithm.


In [27]:
def set_feature_params():
    """Set parameters for Shi-Tomasi corner detection."""
    return dict(maxCorners=100,
                qualityLevel=0.3,
                minDistance=7,
                blockSize=7)

def set_lk_params():
    """Set parameters for Lucas-Kanade optical flow."""
    return dict(winSize=(15, 15),
                maxLevel=2,
                criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))


### Step 4: Get Initial Corners
- The `get_initial_corners` function detects good features to track in the initial frame using the Shi-Tomasi method. This is crucial for establishing the points that will be tracked throughout the video.

In [28]:
def get_initial_corners(old_frame, feature_params):
    """Detect good features to track in the initial frame."""
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    return cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

### Step 5: Process Video Frames
- The `process_frames` function reads frames from the video, calculates the optical flow using the Lucas-Kanade method, and visualizes the results. It draws the tracks of the detected features on the video frames.


In [34]:
def process_frames(cap, feature_params, lk_params):
    """Process video frames to calculate and visualize optical flow."""
    ret, old_frame = cap.read()
    if not ret:
        print('Error: Unable to read the first frame.')
        return

    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)  # Convert the first frame to grayscale
    p0 = get_initial_corners(old_frame, feature_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    while True:
        ret, frame = cap.read()
        if not ret:
            print('No frames grabbed!')
            break

        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # Calculate optical flow
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Draw the tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
                frame = cv.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

            img = cv.add(frame, mask)
            cv.imshow('Optical Flow', img)

        # Update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

        # Exit on ESC key
        if cv.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv.destroyAllWindows()

### Step 6: Display Results
- The processed frames with optical flow visualization are displayed in a window.
- NOTE :  exit the loop by pressing the 'ESC' key


In [35]:

def main(video_path):
    """Main function to run the Lucas-Kanade Optical Flow calculation."""
    cap = initialize_video_capture(video_path)
    feature_params = set_feature_params()
    lk_params = set_lk_params()
    process_frames(cap, feature_params, lk_params)

# Example usage
video_path = 'assets/video.mp4'  # Replace with your video file path
main(video_path)

End

---